In [ ]:

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
Path = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(Path)

### Web Scraping

In [ ]:

all_details= []

for page_number in range(25,1000,25):
	try:

		Path = "C:\Program Files (x86)\chromedriver.exe"
		driver = webdriver.Chrome(Path)
		driver.get(f"https://www.pnet.co.za/5/job-search-detailed.html?what=data%20analyst&where=south%20africa&radius=30&searchorigin=Homepage_top-search&suid=4d67d58b-d00f-4672-b98d-705e2ebb62a0&of={page_number}")
		time.sleep(2)

		# click cookies if they show up
		try:
			coockies = WebDriverWait(driver, 10).until(
       		 EC.presence_of_element_located((By.XPATH, '//div[@id = "ccmgt_explicit_accept"]')))
			coockies.click()
		except:ModuleNotFoundError	

		time.sleep(3) # wait for 3 seconds

		links = [] # create a list to store the links to for each job tab on that page
		job_search = driver.find_elements(by=By.CLASS_NAME, value= "sc-oUcyK")
		
		for i in range(0,24):
				item = job_search[i]
				a = item.find_element(by = By.CLASS_NAME, value = "sc-pYA-dN").find_element(by = By.CLASS_NAME, value = "sc-pbYBj").find_element(by = By.CLASS_NAME, value = "sc-qPlga").find_element(by = By.TAG_NAME, value = "a") .get_property("href")
				links.append(a)

		driver.implicitly_wait(12)

		# Lets loop through each link to access the page of each job
		for link in links:
				try:
					# get one one url
					driver.get(url=link)
					try:
						job_description = driver.find_element(by = By.XPATH, value= '//div[@class = "listing-content-provider-112auwb listingContentBrandingColor at-section-text-description-content"]').text
					except: ModuleNotFoundError
					try:		
						JobTitle = driver.find_element(by = By.XPATH, value= '//h1[@class = "listing__job-title at-header-company-jobTitle sc-gPEVay fPdiNf"]').text
					except: ModuleNotFoundError
					try:
						Contract_type =driver.find_element(by = By.XPATH, value= '//li[@class = "at-listing__list-icons_contract-type sc-kgoBCf cBXldM"]').text
					except: ModuleNotFoundError
					try:
						location = driver.find_element(by = By.XPATH, value= '//li[@class = "at-listing__list-icons_location js-map-offermetadata-link sc-kgoBCf cBXldM"]').text
					except:ModuleNotFoundError
					try:
						job_type = driver.find_element(by = By.XPATH, value= '//li[@class = "at-listing__list-icons_work-type sc-kgoBCf cBXldM"]').text
					except: ModuleNotFoundError
					try:
						date =  driver.find_element(by = By.CSS_SELECTOR, value= 'time').text
					except: ModuleNotFoundError
					try:
						company =  driver.find_element(by = By.CSS_SELECTOR, value= '.jfTQBF').text
					except: ModuleNotFoundError
					
						# take this infomation from each job page
				   
					r = {
						"Job_Title":JobTitle,
						"Company"  : company,
						"Contract Type": Contract_type,
						"Location" : location,
						"date":date,
						"Job Type" :job_type,
						"description":job_description

					}
					all_details.append(r)

					
					time.sleep(3)
				except:ModuleNotFoundError
		time.sleep(3)
	 					
	except:ModuleNotFoundError	
	driver.quit()
	time.sleep(3)
	

					
#create a data frame and then save it as a CSV file		
data = pd.DataFrame(data = all_details)		
data.to_csv("new_dataset.csv")	

In [7]:
#load the original data set
data = pd.read_csv("new_dataset.csv")
# remove duplicates
data = data.drop_duplicates(subset="description") # if two jobs have the same job description, then it must me the same job. the number of entries droped from 232 to 170


### Data Cleaning/Preparation

In [2]:


#load the original data set
data = pd.read_csv("new_dataset.csv")
# remove duplicates
data = data.drop_duplicates(subset="description") # if two jobs have the same job description, then it must me the same job. the number of entries droped from 232 to 170
#convert time from 'x days ago' to date
import datetime

def convert(s):
    parsed_s = [s.split()[:2]]
    time_dict = dict((fmt,float(amount)) for amount,fmt in parsed_s)
    dt = datetime.timedelta(**time_dict)
    past_time = datetime.datetime.now() - dt
    return(str(past_time.date()))
data['date'].replace('1 week ago','7 days ago',inplace = True)
data['date'].replace('1 day ago','1 days ago',inplace = True)
data['date'].replace('1 hour ago','1 hours ago',inplace = True)
data['Date'] = data['date'].apply(convert)
data


#Qualifications to look for
Mathematics = 'mathematics'
statistics = "statistics"
Engineering = "engineering"
computer_science = 'computer science'
information_Tech = 'information technology'
Actuarial_Science = "actuarial science"
BCom  = " bcom"
BSc = ' bsc'
information_S = "information system"

#Skills to look for
SQL_skill = "sql"
python = "python"
R=' r '
power_bi = "power bi"
tableau = "tableau"
Java = "java"
SAS = " sas"
Excel = "excel"
# position
Senior = "senior"
Senior2 = "snr"
Junior = "junior"
Manager = "manager"
Junior2 = "jnr"

#qualification type

post_grad = "post grad"
post_grad2 = "honour"
post_grad3 = "master"

#Job titles i chose for my analysis

DataAnalyst = 'data analyst'
Data_Scientist = 'scien'
Data_Engineer =  'engineer'
Business_Analyst = 'business'
Business_Analyst2 = "bi "
Marketing_analyst = 'market'
operations_analyst = 'operation'
quantitative_analyst = 'quantitative'
Machine_learning_engineer = 'machine'
Test_analyst = 'test'
Actuarial_analyst = 'actuar'
   
# function that extracts Qualifications of interest from the job description
qualifications = [BCom, Mathematics,computer_science,statistics,Engineering,information_Tech,Actuarial_Science,
information_S]
def extract(JobDescriptions):
     return [qualification for qualification in qualifications if qualification in JobDescriptions.lower()]    

#extract skills
Skills = [SQL_skill,python,R,power_bi,SAS, Java,Excel,tableau]
def extract2(JobDescriptions):
     return [skill for skill in Skills if skill in JobDescriptions.lower()]    

#Cartergorize positions 
Positions = [Senior,Junior,Manager,Junior2,Senior2]
def extract3(JobTitle):
     return[position for position in Positions if position in JobTitle.lower()]

#extract qualification type from the job discription
qualification_types = [post_grad,post_grad2,post_grad3]
def extract4(JobDescriptions):
     return [qualification_type for qualification_type in qualification_types if qualification_type in JobDescriptions.lower()]   

# Clean Job titles  
Job_Titles = [Data_Scientist,Data_Engineer,Test_analyst,Business_Analyst,Machine_learning_engineer,DataAnalyst,
               operations_analyst,Marketing_analyst,quantitative_analyst,Business_Analyst2,Actuarial_analyst]
def extract5(Jobtitle):
     return [jobtitle for jobtitle in Job_Titles if jobtitle in Jobtitle.lower()]  

# add these 5 columns by applying the functions defined above
#    1. Qualifications
#    2. Skills
#    3. Qualification Type
#    4. Job Title
#    5. Position

data['qualifications'] = data['description'].apply(extract)
data['skills'] = data['description'].apply(extract2)
data['Position'] = data['Job_Title'].apply(extract3)
data['qualification types'] = data['description'].apply(extract4)
data['Job Title'] = data['Job_Title'].apply(extract5)


# remove list brackets
data['Position'] = data['Position'].str.join(', ')
data['skills'] = data['skills'].str.join(', ')
data['qualifications'] = data['qualifications'].str.join(', ')
data['qualification types' ] = data['qualification types'].str.join(', ')
data['Job Title'] = data['Job Title'].str.join(', ')


# Assign each city to its Metro (we only include JHB, PTA, DBN, Pretoria and Other)
Johannesburg = ['Johannesburg', 'Johannesburg Central', 'Johannesburg East',
       'Midrand', 'JHB - Northern Suburbs','Heidelberg','Gauteng','JHB- Central','Parktown',
       'City of Johannesburg Metropolitan Municipality','Auckland Park','West Rand',
       'JHB - Eastern Suburbs','Randburg', 'Bryanston', 'Sandton','Johannesburg (Woodmead, Sandton)',
       'Germiston', 'Ekurhuleni (East Rand)','Johannesburg West','JHB - Western Suburbs',  'Johannesburg (CBD)',
       'Midrand/Menlyn/Rosslyn/Home Office rotation','Johannesburg - North (Sandton / Midrand / Diepsloot)', 'JHB - Central', 'Johannesburg North',
       'Meadowdale, Gauteng','Southern Suburbs, Cape Town','Roodepoort',
       'Johannesburg (Bruma)','Klerksdorp','Sunninghill', 'Sandton, JHB', 'Sandton, JHB','Johannesburg Metro',]

Pretoria = ['Pretoria','Centurion','Tshwane (Pretoria)','Pretoria East',]

cape_Town = [ 'Cape Town','Cape Town (Partly remote)','Westville','CPT - Northern Suburbs', 'Cape Town: Northern Suburbs',
        'Western Cape','CPT - CBD', 'Cape Town (Partly remote)', 'West Rand / Somerset West',
       'Stellenbosch', 'Belville Park Cape Town' ,'CPT - Southern Suburbs','Cape Town: Helderberg',]
Durban = [ 'Durban', 'Richards Bay','Richards Bay / Johanessburg','Umhlanga','Midrand / Driefontein','DURBAN', 'KZN - North Coast', 'Umhlanga, Durban',
      'Durbanville', ]

other = ['Other',
       'East London', 'Rivonia', 'Nelspruit', 'South Africa', 'Houghton',
       'Coega', 'Pinelands', 'Lichtenburg', 'Mowbray', 'Port Elizabeth',
       'Houghton Estate', 'Kloof', 'South Africa,Johannesburg,Cape Town',
       'Nigeria', 'Saudi Arabia', 'Mount Edgecombe']

Metros = [Johannesburg,cape_Town,Pretoria,Durban,other]
# The method that replaces a city with their metro name
def Replace2(city):
     for metro in Metros:
          if city in metro:
               city = city.replace(city,metro[0])

     return city
# extract the location 
data['Locations'] = data['Location'].apply(Replace2) # add location column to the data frame


#data['Qualification'] = data['Qual IT']+ ', ' + data['qualifications'] # Combine IT qualification and Qualification to one column.
#data['Qualification'].replace( [", "],[" "],inplace=True) 
data = data[['Job Title','Position','qualifications','qualification types','skills','Company','Locations','Contract Type','Job Type','Date']] # choose coolumns to include in a cleaned dataset
# Replace the key Words with their appropriate names
data['Position'].replace( ['jnr','snr',""],['junior','senior',"Mid"],inplace=True) 
data['qualification types'].replace(['post grad','honour','master','','honour, master'],['Post Graduate','Post Graduate','Post Graduate',"Bachelor's Degree",'Post Graduate'],inplace = True)
data['Job Title'].replace(['data analyst','business','bi ','scien','market','quantitative','Machine','operation','test','engineer','actuar'],['Data Analyst','Business Analyst','Business Analyst','Data Scientist','Marketing Analyst','Quantitative Analyst','Machine Learning Engineer','Operations Analyst','Test Analyst','Data Engineer','Actuarial Analyst'],inplace =True)
data['Job Title'].replace(['scien, business','business, data analyst','market, quantitative','business, quantitative', 'engineer, bi ','test, business'],['Data Scientist',"Data Analyst",'Quantitative Analyst','Quantitative Analyst','Data Engineer','Test Analyst'],inplace = True)
data

,Job Title,Position,qualifications,qualification types,skills,Company,Locations,Contract Type,Job Type,Date
0,Data Analyst,Mid,"computer science, information system",Bachelor's Degree,"sql, excel",Express Employment Professionals SA - Midrand,Pretoria,Permanent,Full Time,2022-07-09
1,,junior,,Bachelor's Degree,,Osiris Trading (Pty) Ltd,Johannesburg,Permanent,Full Time,2022-07-04
2,Data Analyst,Mid,,Bachelor's Degree,,Prosperitas Personnel,Johannesburg,Permanent,Full Time,2022-07-04
3,Data Analyst,Mid,,Bachelor's Degree,"sql, power bi, excel",Six Sense Consulting,Johannesburg,Permanent,Full Time,2022-07-04
4,Data Analyst,Mid,,Bachelor's Degree,excel,Time Personnel,Cape Town,Fixed Term,Full Time,2022-07-04
...,...,...,...,...,...,...,...,...,...,...
420,,Mid,"statistics, information technology",Bachelor's Degree,,Indsafri Services SA (Pty) Ltd,Johannesburg,Fixed Term,Full Time,2022-07-04
423,Data Engineer,Mid,,Bachelor's Degree,,eSmart Recruitment CC,Johannesburg,Permanent,Full Time,2022-07-04
425,,Mid,,Bachelor's Degree,,Rainmaker Acquisiton CC,Cape Town,Permanent,Full Time,2022-07-04
427,Data Engineer,Mid,"computer science, statistics, engineering, inf...",Post Graduate,"sql, python",E-Merge IT Recruitment,Johannesburg,Permanent,Full Time,2022-07-04


In [ ]:
data.to_csv("new_dataclean.csv") # save the clean dataset for analysis

### Data Analysis

In [ ]:
new_dataset = data[['qualifications','skills']]
new_dataset

In [6]:
# new df from the column of lists
split_Qualifications = pd.DataFrame(new_dataset['qualifications'].tolist())
split_Qualifications
split_Qualifications.to_csv('qualsplit.csv')

In [7]:
spit_skills = pd.DataFrame(new_dataset['skills'].tolist())
spit_skills
spit_skills.to_csv('SpitSkills.csv')

#### The data was saved and imported to the spreadsheet for easy filterting and analysis